## Figures for Network model of Latent Recurrent Discharge (LRD)


### Some imports and settings


In [ ]:
SAVE_FIGS = False  # see save_args below for configuration
DOWNLOAD_CACHE = False  # if you want the cache files, set to True instead of running simulations again

save_args = dict(use_args=False, close=False, file_formats=["svg", "jpg"])

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings

import brian2.numpy_ as np
import matplotlib.pyplot as plt

from settings import COLOR, G_GABA_LIST, TAU_KCC2_LIST, text, logging, time_unit

# start from defaults before loading stylesheet
plt.style.use("default")
plt.style.use("style/article.mplstyle")

# suppress warnings
np.seterr(all="ignore")
warnings.filterwarnings("ignore")

logger = logging.getLogger("lrd")

In [ ]:
if DOWNLOAD_CACHE:
    %pip install git+https://github.com/ChrisCurrin/dropbox-downloader.git
    import dropbox

    dropbox.downloadFile("https://www.dropbox.com/sh/noqzojddijvvs2a/AAD8ppgjSyi7QIcoGu0e3Ylba?dl=0", ".", unzip=True, retain_zip=False)


# Explain figure


In [ ]:
from scripts.lrdfigure_explain import Explain

np.random.seed(1337)

explain = Explain()

explain.run(
    mv_step=4,
    time_per_value=40,
    egaba=[-78, -34],
    # mv_step=2,
    # time_per_value=60,
    # egaba=[-74, -40],
)

explain.plot()
if SAVE_FIGS:
    explain.save_figure(**save_args)
plt.show()

# Alter conductances

1. Emulate picrotoxin (pink) by **reducing** $g_{GABA_{max}}$.
2. Emulate benzodiazepines (green) by **increasing** $g_{GABA_{max}}$.

We can see the effect of benzodiazepines depends on $E_{GABA}$.


In [ ]:
from scripts.lrdfigure_drugs import Drugs

drugs = Drugs(benzo_strengths=(0, 0.25, 0.5, 1, 2, 4, 8), egabas=[-74, -60, -46])
drugs.run()
drugs.plot(drugs_to_plot=[0.25, 4])
if SAVE_FIGS:
    drugs.save_figure(**save_args)

## Start having dynamic chloride $Cl^-$

We compared networks with either hyperpolarised ($\approx E_{GABA} = -74 mV$) or depolarised ($ \approx E\_{GABA} = -46 mV$) EGABA.

The starting $E_{GABA}$ seems to have neglible, if any effect.

We further consider the effect of GABA conductance $g_{GABA_{max}}$ (brightness, rows) along with different $\tau_{KCC2}$ values (hue, traces).


In [ ]:
from scripts.lrdfigure_chloride import Chloride

cl = Chloride(
    tau_KCC2s=TAU_KCC2_LIST[::2],
    E_Cl_0s=(-60, -88),
    g_GABAs=(50, 25, 100),
    seeds=(None, 1038, 1337, 1111, 1010, 1011, 1101, 1110, 11110, 111100),
)
cl.run(duration=600).process(burst_window=60)
cl.plot(timeit=True, colorbar=False)
if SAVE_FIGS:
    cl.save_figure(**save_args)

In [ ]:
cl.plot(default_tau=60, stripplot_alpha=0.4, stripplot_size=3, stripplot_jitter=0.4)
plt.gcf().set_facecolor("white")

### Supplementary work: the effect of neuron size ($L$) and chloride extrusion ($\tau_{KCC2}$) on the number of bursts in a network


In [ ]:
from scripts.lrdfigure_chloride_length import ChlorideLength

tau_KCC2_list = list(TAU_KCC2_LIST)

ratio = tau_KCC2_list[1] / tau_KCC2_list[0]
tau_KCC2_list = [np.round(tau_KCC2_list[0] / ratio, 1)] + tau_KCC2_list
tau_KCC2_list = [np.round(tau_KCC2_list[0] / ratio, 1)] + tau_KCC2_list
tau_KCC2_list = [np.round(tau_KCC2_list[0] / ratio, 1)] + tau_KCC2_list
tau_KCC2_list = [np.round(tau_KCC2_list[0] / ratio, 1)] + tau_KCC2_list
    # above ratio slightly off but results already cached.
ratio = np.sqrt(2)
tau_KCC2_list = tau_KCC2_list + [np.round(tau_KCC2_list[-1] * ratio, 1)]
tau_KCC2_list = tau_KCC2_list + [np.round(tau_KCC2_list[-1] * ratio, 1)]

cl_length = ChlorideLength(
    tau_KCC2s=tau_KCC2_list[::2],
    g_GABAs=(50, 25, 100),
    lengths=(
        15,
        10,
        7.5,
        6.25,
        5,
    ),
    seeds=(
        None,
        1038,
        1337,
        1111,
        1234,
    ),
)

cl_length.run(duration=600)
cl_length.plot(timeit=True, burst_window=60, colorbar=False)
if SAVE_FIGS:
    cl_length.save_figure(**save_args)

In [ ]:
cl_length.plot(timeit=True, burst_window=60, colorbar=False)
plt.gcf().set_facecolor("white")

# Explore time constants of "Excitatory" and "Inhibitory" KCC2

$\tau_{KCC2_{exc}}$ = [15, 21, 30, 42, 60, 84, 120, 169, 240]

$\tau_{KCC2_{inh}}$ = [15, 21, 30, 42, 60, 84, 120, 169, 240]


In [ ]:
from scripts.lrdfigure_tau import Tau

tau = Tau(
    tau_KCC2_E_list=TAU_KCC2_LIST,
    tau_KCC2_I_list=TAU_KCC2_LIST,
    g_GABA_list=G_GABA_LIST + [37, 75, 150],
    seeds=(
        None,
        1038,
        1337,
        1111,
        1010,
    ),
)

tau.run(duration=600, nrn_idx_i=[0, 1, 2, 3])
tau.process()
tau.plot(
    default_tau_i=60,
    default_tau_e=60,
    default_ggaba=50,
    plot_ggaba=(25, 50, 100, 200),
    with_corner_traces=True,
    run_idx=2,
    use_mean=True,
    square_heatmap=False,
    all_major_ticks=False,
    cmap="viridis",
)
if SAVE_FIGS:
    tau.save_figure(**save_args)

### Old versions

More info, less clarity


In [ ]:
df_num_bursts = tau.df_num_bursts
tau.plot_supp(plot_ggaba=[25, 100], hist_err_bars=True)

## $g_{GABA_{max}}$ vs $\tau_{KCC2}$ with $E_{GABA}$

First, plot number of bursts as a function of $g_{GABA_{\rm{max}}}$ as we directly vary $E_{GABA}$ (static $Cl^-$).

Next, plot number of bursts and $E_{GABA}$ as a function of $g_{GABA_{\rm{max}}}$ and $\tau_{KCC2}$.

Finally, plot the relationship between the pre-burst GABAergic current ($I_{GABA}$) and the number of bursts. The $g_{GABA_{\rm{max}}}$ and $\tau_{KCC2}$ are displayed for reference as color and marker size, respectively. Squares are from static simulations.


In [ ]:
from scripts.lrdfigure_ggaba_e import Gve, mean_igaba, sum_igaba
from style.color import G_GABA_Norm

# add some more lower values for tau
tau_KCC2_list = list(TAU_KCC2_LIST)

ratio = tau_KCC2_list[1] / tau_KCC2_list[0]
tau_KCC2_list = [np.round(tau_KCC2_list[0] / ratio, 1)] + tau_KCC2_list
tau_KCC2_list = [np.round(tau_KCC2_list[0] / ratio, 1)] + tau_KCC2_list
tau_KCC2_list = [np.round(tau_KCC2_list[0] / ratio, 1)] + tau_KCC2_list
tau_KCC2_list = [np.round(tau_KCC2_list[0] / ratio, 1)] + tau_KCC2_list


gve = Gve(
    seeds=(
        None,
        1234,
        5678,
        1426987,
        86751,
        # 1010,
        # 876,
        # 12576,
        # 9681,
        # 814265,
        16928,
        98766,
        876125,
        127658,
        9876,
    ),
    gGABAsvEGABA=sorted(
        set(
            np.append(
                np.round(np.arange(0, 100.0001, 10), 0),
                np.geomspace(10, 1000, 11).round(0),
            )
        )
    ),
    gGABAs=np.geomspace(10, 1000, 11).round(0),
    # gGABAs=sorted(
    #         set(
    #             np.append(
    #                 np.geomspace(10, 1000, 11).round(0),
    #                 np.geomspace(60, 158, 7).round(0),
    #             ))),
    tau_KCC2s=tau_KCC2_list,
)
gve.run()
gve.process()
import seaborn as sns

fig, axes = plt.subplots(ncols=2, figsize=(8, 4))
sns.histplot(
    data=gve.df_g_tau_bursts,
    x=gve.num_bursts_col,
    hue=text.TAU_KCC2,
    multiple="stack",
    palette=COLOR.TAU_SM.cmap,
    # hue_norm=COLOR.TAU_SM.norm,
    hue_norm=G_GABA_Norm(60, 1, 240),
    ax=axes[0],
    alpha=1,
    stat="percent",
)
sns.histplot(
    data=gve.df_g_tau_bursts,
    x=gve.num_bursts_col,
    hue=text.G_GABA,
    multiple="stack",
    palette=COLOR.G_GABA_SM.cmap,
    hue_norm=COLOR.G_GABA_SM.norm,
    ax=axes[1],
    alpha=1,
    stat="percent",
)

# most bursts are <= 15 so for visualisation, set this to the norm max
gve.plot(egabas=5, i_metric=mean_igaba)
if SAVE_FIGS:
    from style.figure import plot_save

    plot_save(f"output/{gve.fig_name}_supp.jpg", figs=[fig], close=False)
    plot_save(f"output/{gve.fig_name}_supp.svg", figs=[fig], close=False)
    gve.save_figure(figs=gve.figs, **save_args)

In [ ]:
import itertools
import pandas as pd

ggaba_ignore = sorted(set(np.geomspace(60, 158, 7).round(0)) - {97})
if "$E_{GABA}$" in gve.df_g_tau_bursts:
    gve.df_g_tau_bursts["EGABA"] = gve.df_g_tau_bursts["$E_{GABA}$"]
df_sub_bursts = gve.df_g_tau_bursts[
    ~gve.df_g_tau_bursts[text.G_GABA].isin(ggaba_ignore)
]
df_sub_tau = gve.df_g_tau[~gve.df_g_tau[text.G_GABA].isin(ggaba_ignore)]

# fill in 0s for missing values
for gaba, tau in itertools.product(
    df_sub_tau[text.G_GABA].unique(), df_sub_tau[text.TAU_KCC2].unique()
):
    entry_exists = (
        df_sub_bursts[
            (df_sub_bursts[text.G_GABA] == gaba) & (df_sub_bursts[text.TAU_KCC2] == tau)
        ].shape[0]
        > 0
    )
    if not entry_exists:
        df_sub_bursts = pd.concat(
            [
                df_sub_bursts,
                pd.DataFrame(
                    {
                        text.G_GABA: gaba,
                        text.TAU_KCC2: tau,
                        gve.num_bursts_col: 0,
                    },
                    index=[0],
                ),
            ],
            ignore_index=True,
        )

fig, axs = plt.subplots(
    2,
    3,
    figsize=(12, 12),
    # sharey="row",
    # gridspec_kw={"width_ratios": [1, 1, 1]},
)
ax = axs[0, 0]

g_gabas = sorted(gve.df_g_E_bursts[text.G_GABA].unique())
egabas = sorted(gve.df_g_E_bursts["EGABA"].unique())
GG, EE = np.meshgrid(g_gabas, egabas)
ZZ = EE

ax.pcolor(GG, EE, ZZ, cmap=COLOR.EGABA_SM.get_cmap(), norm=COLOR.EGABA_SM.norm)
sns.scatterplot(
    data=gve.df_g_E_bursts[
        gve.df_g_E_bursts[text.G_GABA].isin(
            set(gve.df_g_tau_bursts[text.G_GABA].unique())
        )
    ],
    x=text.G_GABA,
    y="EGABA",
    size=gve.num_bursts_col,
    hue=gve.num_bursts_col,
    size_norm=(1, 15),
    hue_norm=(1, 15),
    ax=ax,
    palette="viridis",
    clip_on=False,
    zorder=10,
    # cmap=COLOR.EGABA_SM.get_cmap()
)

ax.set_xscale("log")
ax.set_xlim(10, 1000)

# g_gaba vs EGABA
ax_ggaba = axs[0, 1]

sns.lineplot(
    data=gve.df_g_tau,
    y="EGABA",
    x=text.G_GABA,
    hue=text.TAU_KCC2,
    palette=COLOR.TAU_SM.get_cmap(),
    hue_norm=COLOR.TAU_SM.norm,
    marker=".",
    ax=ax_ggaba,
    legend="full",
)
ax_ggaba.set_xscale("log")


# tau vs EGABA
ax_tau = axs[0, 2]

sns.lineplot(
    data=gve.df_g_tau,
    y="EGABA",
    x=text.TAU_KCC2,
    hue=text.G_GABA,
    palette=COLOR.G_GABA_SM.cmap,
    hue_norm=COLOR.G_GABA_SM.norm,
    marker=".",
    ax=ax_tau,
)
taus = sorted(gve.df_g_tau[text.TAU_KCC2].unique())
ax_tau.set_xscale("log")
tau_labels = [f"{tau:.0f}" if int(tau) == float(tau) else f"{tau}" for tau in taus]
ax_tau.set_xticks(taus, labels=[], minor=True)
ax_tau.set_xticks(taus[::2], labels=tau_labels[::2])

# gve.plot_taukcc2(ax=axs[1,0], cax=axs[0, 1])


sns.scatterplot(
    data=df_sub_bursts,
    y=gve.num_bursts_col,
    x="EGABA",
    ax=axs[1, 0],
    marker=".",
    hue=text.G_GABA,
    palette=COLOR.G_GABA_SM.cmap,
    hue_norm=COLOR.G_GABA_SM.norm,
    size=text.TAU_KCC2,
)
from scipy import stats

reg_data = df_sub_bursts[["EGABA", gve.num_bursts_col]].dropna()
sns.regplot(
    data=reg_data,
    y=gve.num_bursts_col,
    x="EGABA",
    ax=axs[1, 0],
    scatter=False,
    color="k",
)
r = stats.linregress(reg_data["EGABA"], reg_data[gve.num_bursts_col])
print(r)
axs[1, 0].annotate(
    f"$R^2$ = {r.rvalue ** 2:.2f} (p = {r.pvalue:.2g})",
    xy=(0.5, 0.5),
    xytext=(0, 15),
    fontsize="xx-small",
    ha="right",
    va="top",
    color="k",
    # arrowprops=dict(arrowstyle='-|>',connectionstyle="arc3, rad=-0.1")
)
reg_data = reg_data[reg_data["EGABA"] < -50]
sns.regplot(
    data=reg_data,
    y=gve.num_bursts_col,
    x="EGABA",
    ax=axs[1, 0],
    scatter=False,
    color="k",
)
r = stats.linregress(reg_data["EGABA"], reg_data[gve.num_bursts_col])
print(r)
axs[1, 0].annotate(
    f"$R^2$ = {r.rvalue ** 2:.2f} (p = {r.pvalue:.2g})",
    xy=(0.5, 0.5),
    xycoords="axes fraction",
    fontsize="xx-small",
    ha="right",
    va="top",
    # arrowprops=dict(arrowstyle='-|>',connectionstyle="arc3, rad=-0.1")
)

sns.lineplot(
    data=df_sub_bursts,
    y=gve.num_bursts_col,
    x=text.G_GABA,
    hue=text.TAU_KCC2,
    palette=COLOR.TAU_SM.get_cmap(),
    hue_norm=COLOR.TAU_SM.norm,
    ax=axs[1, 1],
    marker=".",
    # multi="stack",
)
# regression
reg_data = df_sub_bursts[[text.G_GABA, gve.num_bursts_col]].dropna()
sns.regplot(
    data=reg_data,
    y=gve.num_bursts_col,
    x=text.G_GABA,
    ax=axs[1, 1],
    scatter=False,
    logx=True,
    color="k",
)
axs[1, 1].set_xscale("log")
r = stats.linregress(reg_data[text.G_GABA], reg_data[gve.num_bursts_col])
print(r)
axs[1, 1].annotate(
    f"$R^2$ = {r.rvalue ** 2:.2f} (p = {r.pvalue:.2g})",
    xy=(0.5, 0.5),
    xycoords="axes fraction",
    fontsize="xx-small",
    ha="right",
    va="top",
)

ax_tau_v_bursts = axs[1, 2]

sns.lineplot(
    data=df_sub_bursts,
    y=gve.num_bursts_col,
    x=text.TAU_KCC2,
    hue=text.G_GABA,
    ax=ax_tau_v_bursts,
    palette=COLOR.G_GABA_SM.cmap,
    hue_norm=COLOR.G_GABA_SM.norm,
    marker=".",
)
# reg
reg_data = df_sub_bursts[[text.TAU_KCC2, gve.num_bursts_col]].dropna()
sns.regplot(
    data=reg_data,
    y=gve.num_bursts_col,
    x=text.TAU_KCC2,
    ax=ax_tau_v_bursts,
    scatter=False,
    logx=True,
    color="k",
)
r = stats.linregress(reg_data[text.TAU_KCC2], reg_data[gve.num_bursts_col])
print(r)
ax_tau_v_bursts.annotate(
    f"$R^2$ = {r.rvalue ** 2:.2f} (p = {r.pvalue:.2g})",
    xy=(0.5, 0.5),
    xycoords="axes fraction",
    fontsize="xx-small",
    ha="right",
    va="top",
)

ax_tau_v_bursts.set_xscale("log")
ax_tau_v_bursts.set_xticks(taus, labels=[], minor=True)
ax_tau_v_bursts.set_xticks(taus[::2], labels=tau_labels[::2])
ax_tau_v_bursts.set_ylim(top=15)
from style.figure import plot_save

# plot_save(f"output/{gve.fig_name}_supp-2.jpg", figs=[fig], close=False)
# plot_save(f"output/{gve.fig_name}_supp-2.svg", figs=[fig], close=False)

In [ ]:
fig, ax = plt.subplots()

sns.histplot(
    data=df_sub_tau,
    # y=gve.num_bursts_col,
    x=text.G_GABA,
    hue=text.TAU_KCC2,
    palette=COLOR.TAU_SM.get_cmap(),
    hue_norm=COLOR.TAU_SM.norm,
    bins=df_sub_tau[text.G_GABA].unique(),
    # discrete=True,binwidth=0.1,
    ax=ax,
    # marker=".",
    # log_scale=True,
    # kde=True,
    multiple="dodge",
    stat="percent",
)
ax.set_xscale("log")

In [ ]:
from matplotlib.colors import Normalize
import numpy as np

g_gaba_range = sorted(df_sub_bursts[text.G_GABA].unique())
tau_kcc2_range = sorted(df_sub_bursts[text.TAU_KCC2].unique())


# Calculate the magnitude of the vector field
fig, ax = plt.subplots()
bursts_max = df_sub_bursts[gve.num_bursts_col].max()
# hue_norm = Normalize(1, bursts_max)
# size_norm = Normalize(1, bursts_max)
sns.scatterplot(
    data=df_sub_bursts[df_sub_bursts[gve.num_bursts_col] > 0],
    y=text.TAU_KCC2,
    x=text.G_GABA,
    hue=gve.num_bursts_col,
    size=gve.num_bursts_col,
    palette="viridis",
    # hue_norm=hue_norm,
    # size_norm=size_norm,
    #  alpha=0.8,
    # set style so that legend has the right marker
    style=gve.num_bursts_col,
    markers="o",
    legend="full",
    ec="w",
    ax=ax,
    zorder=99,
    clip_on=False,
)

# find the line that is the border for 0 bursts
zero_bursts = df_sub_bursts[df_sub_bursts[gve.num_bursts_col] == 0]
max_kcc2_per_gaba = zero_bursts.groupby(text.G_GABA, as_index=False).max()
# ensure every g_gaba has a max_kcc2_per_gaba
lowest_kcc2 = tau_kcc2_range[0]
for g_gaba in g_gaba_range:
    if g_gaba not in max_kcc2_per_gaba[text.G_GABA].unique():
        max_kcc2_per_gaba = pd.concat(
            [
                max_kcc2_per_gaba,
                pd.DataFrame(
                    {
                        text.G_GABA: g_gaba,
                        text.TAU_KCC2: lowest_kcc2,
                    },
                    index=[0],
                ),
            ],
            ignore_index=True,
        )

# plot the line
sns.lineplot(
    data=max_kcc2_per_gaba, x=text.G_GABA, y=text.TAU_KCC2, ax=ax, color="k", zorder=100
)

# Calculate the vector field - the arrow points for each value in df_sub_bursts to the closest point on the max_kcc2_per_gaba line
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Calculate the vector field - the arrow points for each value in df_sub_bursts to the closest point on the max_kcc2_per_gaba line
# note that both dataframes are long-form


u = df_sub_bursts[text.G_GABA] - df_sub_bursts["closest_point"][text.G_GABA]
v = df_sub_bursts[text.TAU_KCC2] - df_sub_bursts["closest_point"][text.TAU_KCC2]

ax: plt.Axes
# Plot the vector field
ax.quiver(
    df_sub_bursts[text.G_GABA],
    df_sub_bursts[text.TAU_KCC2],
    u,
    v,
    scale=1,
    scale_units="xy",
    angles="xy",
    color="blue",
    alpha=0.8,
)


ax.set_xscale("log")
ax.set_yscale("log")
plt.show()

In [ ]:
df_sub_bursts

In [ ]:
import pandas as pd


sns.heatmap(
    data=gve.df_g_tau[pd.isna(gve.df_g_tau["Burst start time (s)"])]
    .pivot_table(
        columns=text.G_GABA,
        index=text.TAU_KCC2,
        values="EGABA",
    )
    .iloc[::-1],
)
# sns.scatterplot(
#     data=gve.df_g_tau[pd.isna(gve.df_g_tau["Burst start time (s)"])],
#     x=text.G_GABA,
#     y=text.TAU_KCC2,
#     size="EGABA",
#     hue="EGABA",
#     hue_order=sorted(gve.df_g_tau["EGABA"].unique(), reverse=True),
#     marker='s',
# )

# plt.gca().set_yscale("log")
# plt.gca().set_xscale("log")

## Supplementary: investigate how excitatory parameters affect the number of bursts.

Specifically, we vary:

1. $g_{AMPA_{\rm{max}}}$
2. $g_{NMDA_{\rm{max}}}$

And run static $Cl^-$ simulations at hyperpolarised (blue), close to $V_{rest}$ (purple), and depolarised (red) EGABA values.


In [ ]:
from scripts.lrdfigure_gampa import Params

exc_params = Params(
    gGABAs=[
        # 0,
        25,
        50,
        100,
        # 200,
    ],
    gAMPAs=np.round(np.arange(0, 20.0001, 5.0), 0),
    gNMDAs=[5.0, 7.5, 10.0],
    seeds=(None, 1013, 12987, 1234, 1837),
)
exc_params.run()
exc_params.process()
exc_params.plot(egabas=[-42, -56, -70])

if SAVE_FIGS:
    exc_params.save_figure(**save_args)

In [ ]:
exc_params.plot_rate_example()

## Supplementary: Number of neurons

> note that displaying the figure inline in jupyter sometimes fails, but saving is successful.


In [ ]:
from scripts.lrdfigure_explain import Explain

np.random.seed(1337)

explain = Explain()

explain.run(
    # mv_step=4,
    # time_per_value=40,
    # egaba=[-78, -34],
    N=5000,
    mv_step=2,
    time_per_value=60,
    egaba=[-74, -40],
)

explain.plot()
if SAVE_FIGS:
    import copy

    supp_save_args = copy.copy(save_args)
    supp_save_args["use_args"] = True
    explain.save_figure(**supp_save_args)
plt.show()

# Playground


`None` keeps default values (see `core.sim.py`)


In [ ]:
from core.sim import single_run

net, results = single_run(
    N=None,
    duration=5 * second,
    dt=None,  # Simulation params
    Mg2_t0=None,
    zero_mag_wash_rate=None,  # Seizure params
    benzo_onset_t=None,
    benzo_wash_rate=None,
    benzo_strength=None,
    benzo_off_t=None,  # drug params
    p=None,
    p_ee=None,
    p_ei=None,
    p_ie=None,
    p_ii=None,  # Connection params
    w=None,
    w_ee=None,
    w_ei=None,
    w_ie=None,
    w_ii=None,  # weight params
    U_0=None,
    tau_d=None,
    tau_f=None,  # STP params
    g_AMPA_max=None,
    g_NMDA_max=None,
    g_GABA_max=None,  # conductances
    E_Cl_0=None,
    E_Cl_target=None,
    E_Cl_end=None,
    E_Cl_pop=None,  # ECl params
    length=None,  # neuron params
    dyn_cl=None,
    manual_cl=None,  # dynamic/manual arg
    tau_KCC2_E=None,
    tau_KCC2_I=None,
    num_ecl_steps=None,  # dynamic + manual params
    __build=True,
    __save_run=False,
    __monitors=None,
    nrn_idx_i=None,  # Brian2 args
    run_seed=None,
    __plot=True,
    __device_directory=None,
)